In [ ]:
def get_movies_since(year):
    import pandas as pd
    year_data = pd.read_csv('year.csv', on_bad_lines='skip', sep="\t")
    movie_data = year_data[year_data["titleType"].isin(["movie", "tvMovie"])]
    movie_data_with_year = movie_data[movie_data["startYear"] >= "1990"]
    movie_data_with_valid_year = movie_data_with_year[movie_data_with_year["startYear"] != "\\N"]
    movie_data_with_valid_year = movie_data_with_valid_year[["tconst", "startYear", "originalTitle"]]

    imdb_data = pd.read_csv("ratings.csv", sep="\t")

    year_vote_data = pd.merge(imdb_data, movie_data_with_valid_year, on="tconst", how="right")
    year_vote_data = year_vote_data[["tconst","startYear","averageRating","numVotes", "originalTitle"]]
    year_vote_data.columns = ["title id","Year","Rating","Votes", "Title"]
    year_vote_data["rating link"] = "https://www.imdb.com/title/"+year_vote_data["title id"]+"/ratings/?ref_=tt_ov_rt"
    year_vote_data = year_vote_data.sort_values(by="title id").reset_index(drop=True)
    year_vote_data.to_csv("movies_since_"+"1990"+".csv", index=False)

    return year_vote_data
dataset = get_movies_since("1990")

## extracting vote data

In [1]:
def get_votes(link, vote_threshold=10000):
    import requests, re
    request_text = requests.get(link).text
    if "No Ratings Available" in request_text or "Well, what if there is no webpage?" in request_text or int(((request_text[request_text.index("IMDb users have given a ")-15:request_text.index("IMDb users have given a ")-1].strip()).replace(",",""))) < vote_threshold:
        return [0,0,0,0,0,0,0,0,0,0], 0.0

    star_votes = re.findall("<div class=\"leftAligned\">(.*?)</div>", request_text)
    rating = re.findall("span class=\"ipl-rating-star__rating\">(.*?)</span>", request_text)
    stars = [int(vote.replace(",","")) for vote in star_votes[1:11]]
    
    return stars, float(rating[0])

In [4]:
# 1 second for a link
stars, rating = get_votes("https://www.imdb.com/title/tt0068646/ratings/?ref_=tt_ov_rt", vote_threshold=10000)

In [2]:
def fill_dataset(dataset, start=0, end=0, zip=1):
    for i in range(start, end):
        try:
            stars, rating = get_votes(dataset.iloc[i]["rating link"])
            dataset.loc[i,["1","2","3","4","5","6","7","8","9","10"]] = stars
            dataset.loc[i,["Rating","Votes"]] = rating, sum(stars)
        except:
            pass
        if i % 100 == 0:
            print(i,". movie done between ",start,"-",end, sep="")

    if zip:
        print(end,". movie done between ",start,"-",end, sep="")
        dataset[:end].to_csv("movie_data_at_"+str(start)+"_"+str(end)+".csv", index=False)
    return dataset

In [ ]:
import pandas as pd
dataset = pd.read_csv("movie_data_at_5000_6000.csv")
dataset = fill_dataset(dataset, start=6000, end=7000, zip=1)

## Merging movie and box office datasets

In [118]:
def merge_movies_with_boxoffices(movie_csv_path, how_to_merge):
    import pandas as pd
    dataset = pd.read_csv(movie_csv_path)
    dataset = dataset[~dataset["1"].isnull()]
    dataset = dataset[dataset["Votes"]>0]
    dataset["Title-Year"] = dataset["Year"].astype(str) + " " + dataset["Title"]

    movie_box_dataset = pd.read_csv("year_movie_worldwide_box.csv")
    movie_box_dataset.columns =  ["Year", "Title", "WorldwideBox Office"]
    movie_box_dataset["Title-Year"] = movie_box_dataset["Year"].astype(int).astype(str) + " " + movie_box_dataset["Title"]

    including_movies = movie_box_dataset[movie_box_dataset["Title"].isin(list(dataset["Title"]))]
    merge = pd.merge(dataset, including_movies[["Title-Year","WorldwideBox Office"]], on="Title-Year", how=how_to_merge)
    return merge

In [ ]:
merge = merge_movies_with_boxoffices("movie_data_at_5000_6000.csv", "outer")
merge